## Create tri-colored PNGs for all slices in HDF5s

In [1]:
import os, sys, re, psycopg2, subprocess
import numpy as np
from subprocess import call

#### Set context

In [2]:
stack_folder = "/Users/OGN/1Projects/ZBrain2/app/assets/images/0-Lines"
channel_folder = "/Users/OGN/1Projects/ZBrain2/app/assets/images/3-ColorChannels/"

os.chdir(stack_folder)
cwd = os.getcwd()
print("Current working directory %s" % cwd)

Current working directory /Users/OGN/1Projects/ZBrain2/app/assets/images/0-Lines


#### Get stack names

In [4]:
stacks = sorted([d for d in os.listdir(stack_folder) if not d.startswith('.')])
print(len(stacks), stacks)

309 ['0-Reference-Brain', '6.7FRhcrtR-Gal4-uasKaede', 'Anti-5HT', 'Anti-Gad67', 'Anti-GlyR', 'Anti-TH', 'Anti-Zn1', 'Anti-Zn12(Hnk-1)', 'Anti-Znp1(Synaptotagmin2)', 'Anti-Zrf1(GFAP)', 'Anti-Zrf2', 'Anti-tERK', 'Elavl3-GCaMP5G', 'Elavl3-H2BRFP', 'EtVmat2-GFP', 'Gad1b-GFP', 'GemaLine_noFish4or3_MeanImageOf5Fish', 'Glyt2-GFP', 'Hcrt-RFP', 'Isl1-GFP', 'Isl2bGal4-uasDendra', 'MH_13319-Gal4', 'MH_15576-Gal4', 'MH_16659-Gal4', 'MH_16715-Gal4', 'MH_16913-Gal4', 'MH_16914-Gal4', 'MH_16915-Gal4', 'MH_16916-Gal4', 'MH_16946-Gal4', 'MH_17056-Gal4', 'MH_17767-Gal4', 'MH_17768-Gal4', 'MH_17776-Gal4', 'MH_17777-Gal4', 'MH_17787-Gal4', 'MH_17845-Gal4', 'MH_17846-Gal4', 'MH_18107-Gal4', 'MH_18108-Gal4', 'MH_18327-Gal4', 'MH_18329-Gal4', 'MH_19322-Gal4', 'MH_19566-Gal4', 'MH_20360-Gal4', 'MH_20361-Gal4', 'MH_20454-Gal4', 'MH_20552-Gal4', 'MH_20553-Gal4', 'MH_20638-Gal4', 'MH_20639-Gal4', 'MH_20640-Gal4', 'MH_20641-Gal4', 'MH_20642-Gal4', 'MH_20643-Gal4', 'MH_20644-Gal4', 'MH_20646-Gal4', 'MH_20647-Gal4'

In [5]:
# Optional, for selecting specific stacks
selector = "Gema"
stacks = [stack for stack in stacks if selector in stack]
print(len(stacks), stacks)

1 ['GemaLine_noFish4or3_MeanImageOf5Fish']


#### Create color channel folders & color subfolders

In [6]:
colors = ['red', 'blue', 'green', 'cyan']
new_stacks = []

for stack in stacks:
    folder_path = "/Users/OGN/1Projects/ZBrain2/app/assets/images/3-ColorChannels/%s/" % stack
    try:
        os.mkdir(folder_path)
        new_stacks.append(stack)
    except:
        pass
    finally:
        for color in colors:
            channel_path = folder_path + color
            try:
                os.mkdir(channel_path)
            except:
                pass

#### Create color channel overlays for each stack (or each new stack depending on input)

In [8]:
for i, stack in enumerate(stacks):
    src = "/Users/OGN/1Projects/ZBrain2/app/assets/images/0-Lines/%s/" % stack
    base = "/Users/OGN/1Projects/ZBrain2/app/assets/images/3-ColorChannels/%s/" % stack
    
    os.chdir(src)
    for color in colors:
        if color is 'green':
            color_val = 'green1'
            fuzz = '7%'
        else:
            color_val = color
            fuzz = '9%'
            
        dest = base + color
        cwd = os.getcwd()
        print(cwd)
        try:
            subprocess.check_output(["mogrify", "-path", dest, "-format", "png", "+level-colors", "black,%s"%color_val, "-fuzz", fuzz, "-transparent", "black", "*.jpg"], stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as err:
            print(err.output)


/Users/OGN/1Projects/ZBrain2/app/assets/images/0-Lines/GemaLine_noFish4or3_MeanImageOf5Fish
/Users/OGN/1Projects/ZBrain2/app/assets/images/0-Lines/GemaLine_noFish4or3_MeanImageOf5Fish
/Users/OGN/1Projects/ZBrain2/app/assets/images/0-Lines/GemaLine_noFish4or3_MeanImageOf5Fish
/Users/OGN/1Projects/ZBrain2/app/assets/images/0-Lines/GemaLine_noFish4or3_MeanImageOf5Fish


### Prepare PostgreSQL table

In [16]:
create_table_command = (
"""
    DROP TABLE colorchannels;
    CREATE TABLE colorChannels (
        channel_id SERIAL PRIMARY KEY,
        channel_color TEXT NOT NULL,
        channel_name TEXT NOT NULL,
        channel_image_path TEXT NOT NULL
    );
""")

#### Connect to zbrain2db database and create table


In [17]:
conn = psycopg2.connect(dbname='zbrain2db', user='gqe', host='localhost', password='')
cursor = conn.cursor()

In [18]:
try:
    cursor.execute(create_table_command)
    cursor.close()
    conn.commit()
except (Exception, psycopg2.DatabaseError) as error:
    print(error)
finally:
    if conn is not None:
        conn.close()

In [19]:
from natsort import natsorted

In [20]:
channel_folder = "/Users/OGN/1Projects/ZBrain2/app/assets/images/3-ColorChannels/"
stack_folder = "/Users/OGN/1Projects/ZBrain2/app/assets/images/0-Lines/" # stacks temporarily stored in folder (0-Lines)
stacks = sorted([d for d in os.listdir(stack_folder) if not d.startswith('.')])
print(stacks)

['0-Reference-Brain', '6.7FRhcrtR-Gal4-uasKaede', 'Anti-5HT', 'Anti-Gad67', 'Anti-GlyR', 'Anti-TH', 'Anti-Zn1', 'Anti-Zn12(Hnk-1)', 'Anti-Znp1(Synaptotagmin2)', 'Anti-Zrf1(GFAP)', 'Anti-Zrf2', 'Anti-tERK', 'Elavl3-GCaMP5G', 'Elavl3-H2BRFP', 'EtVmat2-GFP', 'Gad1b-GFP', 'GemaLine_noFish4or3_MeanImageOf5Fish', 'Glyt2-GFP', 'Hcrt-RFP', 'Isl1-GFP', 'Isl2bGal4-uasDendra', 'MH_13319-Gal4', 'MH_15576-Gal4', 'MH_16659-Gal4', 'MH_16715-Gal4', 'MH_16913-Gal4', 'MH_16914-Gal4', 'MH_16915-Gal4', 'MH_16916-Gal4', 'MH_16946-Gal4', 'MH_17056-Gal4', 'MH_17767-Gal4', 'MH_17768-Gal4', 'MH_17776-Gal4', 'MH_17777-Gal4', 'MH_17787-Gal4', 'MH_17845-Gal4', 'MH_17846-Gal4', 'MH_18107-Gal4', 'MH_18108-Gal4', 'MH_18327-Gal4', 'MH_18329-Gal4', 'MH_19322-Gal4', 'MH_19566-Gal4', 'MH_20360-Gal4', 'MH_20361-Gal4', 'MH_20454-Gal4', 'MH_20552-Gal4', 'MH_20553-Gal4', 'MH_20638-Gal4', 'MH_20639-Gal4', 'MH_20640-Gal4', 'MH_20641-Gal4', 'MH_20642-Gal4', 'MH_20643-Gal4', 'MH_20644-Gal4', 'MH_20646-Gal4', 'MH_20647-Gal4', 'M

#### Create list of table insert commands for each color channel image

In [24]:
def create_insert_commands(stacks):
    channel_id = 0
    for stack in stacks:
        src = "/Users/OGN/1Projects/ZBrain2/app/assets/images/0-Lines/%s/" % stack
        base = "/Users/OGN/1Projects/ZBrain2/app/assets/images/3-ColorChannels/%s/" % stack
        rel = "images/3-ColorChannels/%s/" % stack
    
        colors = ['red', 'blue', 'green', 'cyan']
        for color in colors:
            channel_folder = base + color
            channels = [ c for c in os.listdir(channel_folder) if not c.startswith('.') ]
            channels = [ "%s%s/%s" % (rel, color, channel) for channel in channels]
            channels = natsorted(channels)
            for channel_image_path in channels:
                insert_command = ("INSERT INTO colorChannels (channel_id, channel_color, channel_name, channel_image_path) values (%i, '%s', '%s', '%s')" 
                                 % (channel_id, color, stack, channel_image_path))
                channel_id += 1
                yield insert_command
insert_commands = create_insert_commands(stacks)

#### Iterate insert commands list to create table entries for each color channel image

In [25]:
conn = psycopg2.connect(dbname='zbrain2db', user='gqe', host='localhost', password='')
cursor = conn.cursor()

for insert_command in insert_commands:
    try:
        cursor.execute(insert_command)
    except Exception as e:
        print(e)
        
cursor.close()
conn.commit()

#### Query updated table

In [26]:
conn = psycopg2.connect(dbname='zbrain2db', user='gqe', host='localhost', password='')
cursor = conn.cursor()
cursor.execute('SELECT * FROM colorChannels ORDER BY channel_id')

rows = cursor.fetchall()
for row in rows[0:1000]:
    print(row)

(0, 'red', '0-Reference-Brain', 'images/3-ColorChannels/0-Reference-Brain/red/Reference-Brain-0.png')
(1, 'red', '0-Reference-Brain', 'images/3-ColorChannels/0-Reference-Brain/red/Reference-Brain-1.png')
(2, 'red', '0-Reference-Brain', 'images/3-ColorChannels/0-Reference-Brain/red/Reference-Brain-2.png')
(3, 'red', '0-Reference-Brain', 'images/3-ColorChannels/0-Reference-Brain/red/Reference-Brain-3.png')
(4, 'red', '0-Reference-Brain', 'images/3-ColorChannels/0-Reference-Brain/red/Reference-Brain-4.png')
(5, 'red', '0-Reference-Brain', 'images/3-ColorChannels/0-Reference-Brain/red/Reference-Brain-5.png')
(6, 'red', '0-Reference-Brain', 'images/3-ColorChannels/0-Reference-Brain/red/Reference-Brain-6.png')
(7, 'red', '0-Reference-Brain', 'images/3-ColorChannels/0-Reference-Brain/red/Reference-Brain-7.png')
(8, 'red', '0-Reference-Brain', 'images/3-ColorChannels/0-Reference-Brain/red/Reference-Brain-8.png')
(9, 'red', '0-Reference-Brain', 'images/3-ColorChannels/0-Reference-Brain/red/Refe